In [ ]:
!pip install -U langchain-openai
!pip install -qU langchain
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import Colab Secrets userdata module
from google.colab import userdata

# Set OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Set other API keys similarly
os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

In [ ]:
import time
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(os.environ["PINECONE_API_KEY"])

spec = ServerlessSpec(cloud="aws", region="us-east-1")

index_name = 'fit-pt-rag-test'

# check if index already exists (it shouldn't if this is your first run)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,
        metric='euclidean',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 492}},
 'total_vector_count': 492}

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://www.healthcoachinstitute.com/articles/24-workout-motivational-quotes/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

len(documents)

21

In [ ]:
from langchain.vectorstores import  Pinecone
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
docsearch = Pinecone.from_texts([documents[0].page_content for t in docs], embeddings, index_name=index_name)

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 492}},
 'total_vector_count': 492}

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model_name='gpt-3.5-turbo',
    temperature=0.0)

retriever = docsearch.as_retriever()
docs = retriever.invoke("question")

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="excercise", description="The list of exercises for a workout plan"),
    ResponseSchema(name="journal", description="The questions for the fitness journal",)]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

format_instructions = output_parser.get_format_instructions()

# Prompt
prompt = ChatPromptTemplate.from_template(
    template="""You are a Personal Fitness Trainer. First Generate a detailed list of exercises that will be performed that day.
    Use the context retrieved to create the workout and ask the client questions about the workout to add to a Fitness Journal. {context}
    Your task is to generate a workout based on the location the client is using.
    Location: {location}
    {format_instructions}
    """, partial_variables={"format_instructions": format_instructions},)

# LLM
chain = prompt | llm | output_parser


In [ ]:
# Run
answer = chain.invoke({"context":docs,"location":input("Where are you working out today?: ")})
answer

Where are you working out today?: I am going to a gym, I want to focus on upper body


{'excercise': '1. Bench Press - 4 sets of 10 reps\n2. Pull-ups - 3 sets of 12 reps\n3. Dumbbell Shoulder Press - 3 sets of 15 reps\n4. Bent Over Rows - 4 sets of 12 reps\n5. Tricep Dips - 3 sets of 15 reps\n6. Bicep Curls - 3 sets of 12 reps',
 'journal': '1. How did you feel during the workout? \n2. Did you struggle with any particular exercise? \n3. Did you feel any muscle fatigue or soreness during or after the workout? \n4. How would you rate your overall performance and effort during the workout? \n5. Did you enjoy focusing on the upper body today?'}

In [ ]:
exercise = answer['excercise']
workout = exercise.split('\n')
workout

['1. Bench Press - 4 sets of 10 reps',
 '2. Pull-ups - 3 sets of 12 reps',
 '3. Dumbbell Shoulder Press - 3 sets of 15 reps',
 '4. Bent Over Rows - 4 sets of 12 reps',
 '5. Tricep Dips - 3 sets of 15 reps',
 '6. Bicep Curls - 3 sets of 12 reps']

In [ ]:
journal_questions = answer['journal']
questions = journal_questions.split('\n')
questions

['1. How did you feel during the workout? ',
 '2. Did you struggle with any particular exercise? ',
 '3. Did you feel any muscle fatigue or soreness during or after the workout? ',
 '4. How would you rate your overall performance and effort during the workout? ',
 '5. Did you enjoy focusing on the upper body today?']

In [ ]:
# Create a dictionary to store the user's responses
responses = {}

# Iterate over each question and prompt the user for an answer
for question in questions:
    print(question)
    response = input("Your answer: ")
    responses[question] = response

# Optionally, display all responses
for question, response in responses.items():
    print(f"{question}, {response}")


1. How did you feel during the workout? 
Your answer: Okay, I have a food injury so this was tough
2. Did you struggle with any particular exercise? 
Your answer: I don't like pull ups
3. Did you feel any muscle fatigue or soreness during or after the workout? 
Your answer: just awkwardness from walking weird
4. How would you rate your overall performance and effort during the workout? 
Your answer: Maybe a 7, just because I pushed myself through
5. Did you enjoy focusing on the upper body today?
Your answer: I did, I like lower body so this was new
1. How did you feel during the workout? , Okay, I have a food injury so this was tough
2. Did you struggle with any particular exercise? , I don't like pull ups
3. Did you feel any muscle fatigue or soreness during or after the workout? , just awkwardness from walking weird
4. How would you rate your overall performance and effort during the workout? , Maybe a 7, just because I pushed myself through
5. Did you enjoy focusing on the upper bo